# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fa4d8ae1e80>
├── 'a' --> tensor([[ 0.4259,  0.1133, -0.6068],
│                   [-0.7104,  0.3368,  0.0771]])
└── 'x' --> <FastTreeValue 0x7fa4d8ae1dc0>
    └── 'c' --> tensor([[-1.2889,  0.5265,  0.4927,  0.7358],
                        [-0.4130, -0.2697, -0.7697, -1.5057],
                        [ 0.1167,  1.8936,  0.5059, -0.1902]])

In [4]:
t.a

tensor([[ 0.4259,  0.1133, -0.6068],
        [-0.7104,  0.3368,  0.0771]])

In [5]:
%timeit t.a

66.1 ns ± 0.017 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fa4d8ae1e80>
├── 'a' --> tensor([[ 0.1541, -0.1861,  1.3594],
│                   [-1.2370,  0.8547,  0.9108]])
└── 'x' --> <FastTreeValue 0x7fa4d8ae1dc0>
    └── 'c' --> tensor([[-1.2889,  0.5265,  0.4927,  0.7358],
                        [-0.4130, -0.2697, -0.7697, -1.5057],
                        [ 0.1167,  1.8936,  0.5059, -0.1902]])

In [7]:
%timeit t.a = new_value

67.7 ns ± 0.07 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.4259,  0.1133, -0.6068],
               [-0.7104,  0.3368,  0.0771]]),
    x: Batch(
           c: tensor([[-1.2889,  0.5265,  0.4927,  0.7358],
                      [-0.4130, -0.2697, -0.7697, -1.5057],
                      [ 0.1167,  1.8936,  0.5059, -0.1902]]),
       ),
)

In [10]:
b.a

tensor([[ 0.4259,  0.1133, -0.6068],
        [-0.7104,  0.3368,  0.0771]])

In [11]:
%timeit b.a

60.2 ns ± 0.0837 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.3487,  0.2407, -0.2800],
               [-0.3865, -1.9855, -0.6119]]),
    x: Batch(
           c: tensor([[-1.2889,  0.5265,  0.4927,  0.7358],
                      [-0.4130, -0.2697, -0.7697, -1.5057],
                      [ 0.1167,  1.8936,  0.5059, -0.1902]]),
       ),
)

In [13]:
%timeit b.a = new_value

493 ns ± 0.0952 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

876 ns ± 1.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 9.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 821 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 1.17 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fa4d8b39e50>
├── 'a' --> tensor([[[ 0.4259,  0.1133, -0.6068],
│                    [-0.7104,  0.3368,  0.0771]],
│           
│                   [[ 0.4259,  0.1133, -0.6068],
│                    [-0.7104,  0.3368,  0.0771]],
│           
│                   [[ 0.4259,  0.1133, -0.6068],
│                    [-0.7104,  0.3368,  0.0771]],
│           
│                   [[ 0.4259,  0.1133, -0.6068],
│                    [-0.7104,  0.3368,  0.0771]],
│           
│                   [[ 0.4259,  0.1133, -0.6068],
│                    [-0.7104,  0.3368,  0.0771]],
│           
│                   [[ 0.4259,  0.1133, -0.6068],
│                    [-0.7104,  0.3368,  0.0771]],
│           
│                   [[ 0.4259,  0.1133, -0.6068],
│                    [-0.7104,  0.3368,  0.0771]],
│           
│                   [[ 0.4259,  0.1133, -0.6068],
│                    [-0.7104,  0.3368,  0.0771]]])
└── 'x' --> <FastTreeValue 0x7fa4d8b39dc0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.7 µs ± 64.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fa4d8ad4130>
├── 'a' --> tensor([[ 0.4259,  0.1133, -0.6068],
│                   [-0.7104,  0.3368,  0.0771],
│                   [ 0.4259,  0.1133, -0.6068],
│                   [-0.7104,  0.3368,  0.0771],
│                   [ 0.4259,  0.1133, -0.6068],
│                   [-0.7104,  0.3368,  0.0771],
│                   [ 0.4259,  0.1133, -0.6068],
│                   [-0.7104,  0.3368,  0.0771],
│                   [ 0.4259,  0.1133, -0.6068],
│                   [-0.7104,  0.3368,  0.0771],
│                   [ 0.4259,  0.1133, -0.6068],
│                   [-0.7104,  0.3368,  0.0771],
│                   [ 0.4259,  0.1133, -0.6068],
│                   [-0.7104,  0.3368,  0.0771],
│                   [ 0.4259,  0.1133, -0.6068],
│                   [-0.7104,  0.3368,  0.0771]])
└── 'x' --> <FastTreeValue 0x7fa4d8ad4310>
    └── 'c' --> tensor([[-1.2889,  0.5265,  0.4927,  0.7358],
                        [-0.4130, -0.2697, -0.7697, -1.5057],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 53.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.6 µs ± 177 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.4259,  0.1133, -0.6068],
                [-0.7104,  0.3368,  0.0771]],
       
               [[ 0.4259,  0.1133, -0.6068],
                [-0.7104,  0.3368,  0.0771]],
       
               [[ 0.4259,  0.1133, -0.6068],
                [-0.7104,  0.3368,  0.0771]],
       
               [[ 0.4259,  0.1133, -0.6068],
                [-0.7104,  0.3368,  0.0771]],
       
               [[ 0.4259,  0.1133, -0.6068],
                [-0.7104,  0.3368,  0.0771]],
       
               [[ 0.4259,  0.1133, -0.6068],
                [-0.7104,  0.3368,  0.0771]],
       
               [[ 0.4259,  0.1133, -0.6068],
                [-0.7104,  0.3368,  0.0771]],
       
               [[ 0.4259,  0.1133, -0.6068],
                [-0.7104,  0.3368,  0.0771]]]),
    x: Batch(
           c: tensor([[[-1.2889,  0.5265,  0.4927,  0.7358],
                       [-0.4130, -0.2697, -0.7697, -1.5057],
                       [ 0.1167,  1.8936,  0.5059, -0.1902]],
         

In [26]:
%timeit Batch.stack(batches)

77.2 µs ± 153 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.4259,  0.1133, -0.6068],
               [-0.7104,  0.3368,  0.0771],
               [ 0.4259,  0.1133, -0.6068],
               [-0.7104,  0.3368,  0.0771],
               [ 0.4259,  0.1133, -0.6068],
               [-0.7104,  0.3368,  0.0771],
               [ 0.4259,  0.1133, -0.6068],
               [-0.7104,  0.3368,  0.0771],
               [ 0.4259,  0.1133, -0.6068],
               [-0.7104,  0.3368,  0.0771],
               [ 0.4259,  0.1133, -0.6068],
               [-0.7104,  0.3368,  0.0771],
               [ 0.4259,  0.1133, -0.6068],
               [-0.7104,  0.3368,  0.0771],
               [ 0.4259,  0.1133, -0.6068],
               [-0.7104,  0.3368,  0.0771]]),
    x: Batch(
           c: tensor([[-1.2889,  0.5265,  0.4927,  0.7358],
                      [-0.4130, -0.2697, -0.7697, -1.5057],
                      [ 0.1167,  1.8936,  0.5059, -0.1902],
                      [-1.2889,  0.5265,  0.4927,  0.7358],
                      [-0.4130, -

In [28]:
%timeit Batch.cat(batches)

144 µs ± 153 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

324 µs ± 2.61 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
